This notebook uses Python to review the basics of $\chi^2$ tests. They are a way to test for significance for categorical variables.

I would also like to more completely understand the critical values used to compare $\chi^2$ values depending on the degrees of freedom.

In [3]:
#packages used in this notebook
import random
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
plt.style.use("ggplot")

In [6]:
len([1,2,3,5,2,3,4,4,1,2,3,4,3,5,2,1,3,4,5])

19

In [7]:
#create a dataframe with categorical variables
newframe = pd.DataFrame({"catvar1":[1,2,3,5,2,3,4,4,1,2,3,4,3,5,2,1,3,4,5]},index=range(19))